In [6]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pandas tensorflow keras os numpy tqdm

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import re
from tqdm import tqdm
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import (
    Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, 
    TextVectorization, BatchNormalization, SpatialDropout1D
)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns

# Tắt warning
import warnings
warnings.filterwarnings('ignore')

# Sử dụng GPU nếu có
print("Kiểm tra GPU...")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")
if tf.config.list_physical_devices('GPU'):
    print("✓ Đang sử dụng GPU")
else:
    print("⚠ Đang sử dụng CPU")

# --- 1. Tải dữ liệu ---
def load_data_from_txt(good_file, bad_file):
    """Tải dữ liệu từ file txt"""
    good_urls = []
    print("\n📁 Đang đọc file URL sạch...")
    with open(good_file, 'r', encoding='utf-8', errors='ignore') as f:
        for line in tqdm(f, desc=f"Đọc {good_file}"):
            url = line.strip()
            if url:  # Bỏ qua dòng trống
                good_urls.append(url)

    bad_urls = []
    print("\n📁 Đang đọc file URL độc hại...")
    with open(bad_file, 'r', encoding='utf-8', errors='ignore') as f:
        for line in tqdm(f, desc=f"Đọc {bad_file}"):
            url = line.strip()
            if url:  # Bỏ qua dòng trống
                bad_urls.append(url)

    df_good = pd.DataFrame({'url': good_urls, 'label': 0})
    df_bad = pd.DataFrame({'url': bad_urls, 'label': 1})

    df = pd.concat([df_good, df_bad], ignore_index=True)
    
    # Loại bỏ duplicate
    before = len(df)
    df = df.drop_duplicates(subset=['url'])
    after = len(df)
    if before != after:
        print(f"✓ Đã loại bỏ {before - after} URL trùng lặp")
    
    return df

# --- 2. Tiền xử lý ---
def preprocess_url_for_keras(url):
    """
    Tiền xử lý URL cho mô hình Keras.
    Tách mỗi ký tự bằng dấu cách để TextVectorization xử lý từng ký tự.
    """
    # Chuyển về lowercase
    url = url.lower()
    # Xóa http://, https://, www.
    url = re.sub(r'https?://|www\.', '', url)
    # Giới hạn độ dài (tránh URL quá dài)
    url = url[:500]
    # Tách mọi ký tự bằng dấu cách
    url = " ".join(list(url))
    return url

# --- 3. Xây dựng mô hình CNN tối ưu ---
def build_optimized_model(vocab_size=128, max_len=200, embedding_dim=64):
    """
    Xây dựng mô hình CNN 1D tối ưu với:
    - Batch Normalization
    - Multiple Conv layers
    - Spatial Dropout
    - Better regularization
    """
    # Input layer
    input_layer = Input(shape=(1,), dtype=tf.string, name='input_url')
    
    # TextVectorization layer (sẽ được adapt sau)
    vectorize_layer = TextVectorization(
        max_tokens=vocab_size,
        output_mode='int',
        output_sequence_length=max_len
    )
    
    # Embedding layer
    x = vectorize_layer(input_layer)
    x = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        input_length=max_len,
        mask_zero=True  # Hỗ trợ padding
    )(x)
    
    # Spatial Dropout (tốt hơn cho embedding)
    x = SpatialDropout1D(0.2)(x)
    
    # Multiple Conv1D layers với kernel sizes khác nhau
    conv1 = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(x)
    conv1 = BatchNormalization()(conv1)
    
    conv2 = Conv1D(filters=128, kernel_size=5, activation='relu', padding='same')(x)
    conv2 = BatchNormalization()(conv2)
    
    conv3 = Conv1D(filters=128, kernel_size=7, activation='relu', padding='same')(x)
    conv3 = BatchNormalization()(conv3)
    
    # Concatenate các conv layers
    x = tf.keras.layers.Concatenate()([conv1, conv2, conv3])
    
    # Global pooling
    x = GlobalMaxPooling1D()(x)
    
    # Dense layers
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    # Output layer
    output_layer = Dense(1, activation='sigmoid', name='output')(x)
    
    # Tạo model
    model = Model(inputs=input_layer, outputs=output_layer)
    
    return model, vectorize_layer

# --- 4. Vẽ biểu đồ training history ---
def plot_training_history(history, save_path='training_history.png'):
    """Vẽ biểu đồ loss và accuracy"""
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Loss
    axes[0].plot(history.history['loss'], label='Train Loss')
    axes[0].plot(history.history['val_loss'], label='Val Loss')
    axes[0].set_title('Model Loss', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Accuracy
    axes[1].plot(history.history['accuracy'], label='Train Accuracy')
    axes[1].plot(history.history['val_accuracy'], label='Val Accuracy')
    axes[1].set_title('Model Accuracy', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Đã lưu biểu đồ training tại: {save_path}")
    plt.close()

# --- 5. Vẽ confusion matrix ---
def plot_confusion_matrix(y_true, y_pred, save_path='confusion_matrix.png'):
    """Vẽ confusion matrix"""
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Sạch', 'Độc hại'],
                yticklabels=['Sạch', 'Độc hại'])
    plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Đã lưu confusion matrix tại: {save_path}")
    plt.close()

# --- 6. Hàm chính ---
def main_process_and_train_keras():
    print("\n" + "="*80)
    print(" "*20 + "URL CLASSIFIER - TRAINING PIPELINE")
    print("="*80)
    
    # --- Tải dữ liệu ---
    print("\n📊 BƯỚC 1: LOAD DỮ LIỆU")
    print("-"*80)
    
    try:
        df = load_data_from_txt('benign.txt', 'malicious.txt')
    except FileNotFoundError as e:
        print(f"❌ Lỗi: Không tìm thấy file - {e}")
        return

    df_good = df[df['label'] == 0]
    df_bad = df[df['label'] == 1]
    
    print(f"\n✓ Tổng số URL: {len(df):,}")
    print(f"  - URL Sạch (0): {len(df_good):,}")
    print(f"  - URL Độc hại (1): {len(df_bad):,}")
    
    # --- Cân bằng dữ liệu ---
    print("\n⚖️ BƯỚC 2: CÂN BẰNG DỮ LIỆU")
    print("-"*80)
    
    # Tự động điều chỉnh N_SAMPLES
    N_SAMPLES = min(len(df_good), len(df_bad))
    
    # Để lại 20% cho test
    train_val_size = int(N_SAMPLES * 0.8)
    test_size = N_SAMPLES - train_val_size
    
    print(f"✓ Sử dụng {train_val_size:,} mẫu/loại cho Train+Val")
    print(f"✓ Sử dụng {test_size:,} mẫu/loại cho Test")
    
    # Chia dữ liệu
    good_train_val = df_good.sample(n=train_val_size, random_state=42)
    bad_train_val = df_bad.sample(n=train_val_size, random_state=42)
    
    good_test = df_good.drop(good_train_val.index).sample(n=test_size, random_state=42)
    bad_test = df_bad.drop(bad_train_val.index).sample(n=test_size, random_state=42)
    
    df_train_val = pd.concat([good_train_val, bad_train_val]).sample(frac=1, random_state=42)
    df_test = pd.concat([good_test, bad_test]).sample(frac=1, random_state=42)
    
    print(f"\n✓ Train/Val: {len(df_train_val):,} samples")
    print(f"✓ Test: {len(df_test):,} samples")
    
    # --- Tiền xử lý ---
    print("\n🔧 BƯỚC 3: TIỀN XỬ LÝ DỮ LIỆU")
    print("-"*80)
    
    tqdm.pandas(desc="Xử lý Train/Val")
    df_train_val['processed_url'] = df_train_val['url'].progress_apply(preprocess_url_for_keras)
    
    tqdm.pandas(desc="Xử lý Test")
    df_test['processed_url'] = df_test['url'].progress_apply(preprocess_url_for_keras)
    
    # --- Chia Train/Val ---
    print("\n✂️ BƯỚC 4: CHIA TRAIN/VALIDATION")
    print("-"*80)
    
    X_tv = df_train_val['processed_url'].values
    y_tv = df_train_val['label'].values.astype('float32')
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_tv, y_tv, test_size=0.2, random_state=42, stratify=y_tv
    )
    
    X_test = df_test['processed_url'].values
    y_test = df_test['label'].values.astype('float32')
    
    print(f"✓ Train: {len(X_train):,} samples")
    print(f"✓ Validation: {len(X_val):,} samples")
    print(f"✓ Test: {len(X_test):,} samples")
    
    # --- Xây dựng Model ---
    print("\n🏗️ BƯỚC 5: XÂY DỰNG MÔ HÌNH")
    print("-"*80)
    
    MAX_LEN = 200
    VOCAB_SIZE = 128
    EMBEDDING_DIM = 64
    
    model, vectorize_layer = build_optimized_model(
        vocab_size=VOCAB_SIZE,
        max_len=MAX_LEN,
        embedding_dim=EMBEDDING_DIM
    )
    
    # Adapt vectorization layer
    print("\n✓ Đang adapt TextVectorization layer...")
    vectorize_layer.adapt(X_train)
    print("✓ Adapt hoàn tất")
    
    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy', 
                 tf.keras.metrics.Precision(name='precision'),
                 tf.keras.metrics.Recall(name='recall')]
    )
    
    print("\n📋 Kiến trúc mô hình:")
    model.summary()
    
    # --- Callbacks ---
    print("\n⚙️ BƯỚC 6: CÀI ĐẶT CALLBACKS")
    print("-"*80)
    
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=2,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            'best_modelsaved.keras',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )
    ]
    
    print("✓ EarlyStopping (patience=3)")
    print("✓ ReduceLROnPlateau (factor=0.5, patience=2)")
    print("✓ ModelCheckpoint (save best model)")
    
    # --- Tính Class Weight ---
    classes = np.array([0, 1])
    class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
    class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
    print(f"\n✓ Class Weights: {class_weight_dict}")
    
    # --- Training ---
    print("\n🚀 BƯỚC 7: HUẤN LUYỆN MÔ HÌNH")
    print("-"*80)
    
    N_EPOCHS = 20
    BATCH_SIZE = 512  # Tối ưu cho cả CPU và GPU
    
    print(f"✓ Epochs: {N_EPOCHS}")
    print(f"✓ Batch Size: {BATCH_SIZE}")
    print("\nBắt đầu training...\n")
    
    history = model.fit(
        X_train,
        y_train,
        epochs=N_EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(X_val, y_val),
        class_weight=class_weight_dict,
        callbacks=callbacks,
        verbose=1
    )
    
    print("\n✓ Training hoàn tất!")
    
    # --- Vẽ biểu đồ ---
    print("\n📊 BƯỚC 8: VẼ BIỂU ĐỒ")
    print("-"*80)
    plot_training_history(history)
    
    # --- Lưu Model ---
    print("\n💾 BƯỚC 9: LƯU MÔ HÌNH")
    print("-"*80)
    
    MODEL_PATH = 'url_classifier_model.keras'
    model.save(MODEL_PATH)
    print(f"✓ Đã lưu model (dạng file Keras) tại: {MODEL_PATH}")
    
    # --- Đánh giá trên Test ---
    print("\n🎯 BƯỚC 10: ĐÁNH GIÁ TRÊN TẬP TEST")
    print("-"*80)
    
    print("\nĐang đánh giá...")
    test_results = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=0)
    
    print("\n📈 Kết quả trên tập Test:")
    print(f"  - Loss: {test_results[0]:.4f}")
    print(f"  - Accuracy: {test_results[1]*100:.2f}%")
    print(f"  - Precision: {test_results[2]*100:.2f}%")
    print(f"  - Recall: {test_results[3]*100:.2f}%")
    
    # Classification Report
    print("\nĐang tạo dự đoán cho báo cáo chi tiết...")
    y_test_pred_proba = model.predict(X_test, batch_size=BATCH_SIZE, verbose=0)
    y_test_pred = (y_test_pred_proba > 0.5).astype(int).flatten()
    
    print("\n📋 Classification Report:")
    print("-"*80)
    print(classification_report(
        y_test, 
        y_test_pred, 
        target_names=['Sạch (0)', 'Độc hại (1)'],
        digits=4
    ))
    
    # Confusion Matrix
    plot_confusion_matrix(y_test, y_test_pred)
    
    # --- Hoàn thành ---
    print("\n" + "="*80)
    print("✅ HOÀN THÀNH! Mô hình đã sẵn sàng để sử dụng.")
    print("="*80)
    print(f"\n📁 Files đã tạo:")
    print(f"  - {MODEL_PATH} (mô hình chính)")
    print(f"  - best_model.h5 (mô hình tốt nhất)")
    print(f"  - training_history.png (biểu đồ training)")
    print(f"  - confusion_matrix.png (ma trận nhầm lẫn)")
    print("\n💡 Để test mô hình, sử dụng file test script với model này!")

# --- Main ---
if __name__ == "__main__":
    main_process_and_train_keras()

Kiểm tra GPU...
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
✓ Đang sử dụng GPU

                    URL CLASSIFIER - TRAINING PIPELINE

📊 BƯỚC 1: LOAD DỮ LIỆU
--------------------------------------------------------------------------------

📁 Đang đọc file URL sạch...


Đọc benign.txt: 6012821it [00:01, 4145059.53it/s]



📁 Đang đọc file URL độc hại...


Đọc malicious.txt: 3524939it [00:00, 4160118.45it/s]


✓ Đã loại bỏ 39896 URL trùng lặp

✓ Tổng số URL: 9,497,864
  - URL Sạch (0): 6,012,821
  - URL Độc hại (1): 3,485,043

⚖️ BƯỚC 2: CÂN BẰNG DỮ LIỆU
--------------------------------------------------------------------------------
✓ Sử dụng 2,788,034 mẫu/loại cho Train+Val
✓ Sử dụng 697,009 mẫu/loại cho Test

✓ Train/Val: 5,576,068 samples
✓ Test: 1,394,018 samples

🔧 BƯỚC 3: TIỀN XỬ LÝ DỮ LIỆU
--------------------------------------------------------------------------------


...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_url           │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ text_vectorization… │ (None, 200)       │          0 │ input_url[0][0]   │
│ (TextVectorization) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 200, 64)   │      8,192 │ text_vectorizati… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d_1 │ (None, 200, 64)   │          0 │ embedding_1[0][0] │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 200, 128)  │     24,704 │ spatial_dropout1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 200, 128)  │     41,088 │ spatial_dropout1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 200, 128)  │     57,472 │ spatial_dropout1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 200, 128)  │        512 │ conv1d_3[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 200, 128)  │        512 │ conv1d_4[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 200, 128)  │        512 │ conv1d_5[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 200, 384)  │          0 │ batch_normalizat… │
│ (Concatenate)       │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 384)       │          0 │ concatenate_1[0]… │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │     98,560 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256)       │      1,024 │ dense_2[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 256)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │     32,896 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_3[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 128)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │        129 │ dropout_3[0][0] 

 Total params: 266,113 (1.02 MB)

 Trainable params: 264,577 (1.01 MB)

 Non-trainable params: 1,536 (6.00 KB)


⚙️ BƯỚC 6: CÀI ĐẶT CALLBACKS
--------------------------------------------------------------------------------
✓ EarlyStopping (patience=3)
✓ ReduceLROnPlateau (factor=0.5, patience=2)
✓ ModelCheckpoint (save best model)

✓ Class Weights: {0: 1.0, 1: 1.0}

🚀 BƯỚC 7: HUẤN LUYỆN MÔ HÌNH
--------------------------------------------------------------------------------
✓ Epochs: 20
✓ Batch Size: 512

Bắt đầu training...

Epoch 1/20
8713/8713 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.9613 - loss: 0.1100 - precision: 0.9727 - recall: 0.9493
Epoch 1: val_accuracy improved from None to 0.97732, saving model to best_modelsaved.keras
8713/8713 ━━━━━━━━━━━━━━━━━━━━ 464s 53ms/step - accuracy: 0.9691 - loss: 0.0928 - precision: 0.9797 - recall: 0.9581 - val_accuracy: 0.9773 - val_loss: 0.0730 - val_precision: 0.9872 - val_recall: 0.9672 - learning_rate: 0.0010
Epoch 2/20
8712/8713 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.9753 - loss: 0.0780 - precision: 0.9845 - recall: 0.9658
Epoch 2: 

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix,
    roc_curve, roc_auc_score,
    precision_recall_curve, average_precision_score
)
from tqdm import tqdm
import re
import warnings
warnings.filterwarnings('ignore')

# ==========================================
# 1. LOAD MODEL
# ==========================================
print("\n📦 BƯỚC 1: LOAD MODEL")
print("-"*80)

MODEL_PATH = 'url_classifier_model.keras'  # Hoặc 'best_modelsaved.keras'

try:
    model = keras.models.load_model(MODEL_PATH)
    print(f"✅ Đã load model từ: {MODEL_PATH}")
    print(f"✓ Model architecture:")
    model.summary()
except Exception as e:
    print(f"❌ Lỗi load model: {e}")
    print("💡 Kiểm tra lại đường dẫn file model!")
    exit(1)

# ==========================================
# 2. LOAD VÀ TIỀN XỬ LÝ DỮ LIỆU TEST
# ==========================================
print("\n📊 BƯỚC 2: LOAD DỮ LIỆU TEST")
print("-"*80)

def preprocess_url_for_keras(url):
    """Tiền xử lý URL (giống hệt code train)"""
    url = url.lower()
    url = re.sub(r'https?://|www\.', '', url)
    url = url[:500]
    url = " ".join(list(url))
    return url

def load_data_from_txt(good_file, bad_file):
    """Tải dữ liệu từ file txt"""
    good_urls = []
    print(f"\n📁 Đang đọc {good_file}...")
    with open(good_file, 'r', encoding='utf-8', errors='ignore') as f:
        for line in tqdm(f, desc="Đọc URL sạch"):
            url = line.strip()
            if url:
                good_urls.append(url)

    bad_urls = []
    print(f"\n📁 Đang đọc {bad_file}...")
    with open(bad_file, 'r', encoding='utf-8', errors='ignore') as f:
        for line in tqdm(f, desc="Đọc URL độc hại"):
            url = line.strip()
            if url:
                bad_urls.append(url)

    df_good = pd.DataFrame({'url': good_urls, 'label': 0})
    df_bad = pd.DataFrame({'url': bad_urls, 'label': 1})
    df = pd.concat([df_good, df_bad], ignore_index=True)
    df = df.drop_duplicates(subset=['url'])
    
    return df

# Load data
try:
    df = load_data_from_txt('benign.txt', 'malicious.txt')
    
    # ⚠️ QUAN TRỌNG: Sort để đảm bảo thứ tự giống code train
    df = df.sort_values('url').reset_index(drop=True)
    
    # Lấy test set (20% cuối cùng như code train)
    df_good = df[df['label'] == 0].reset_index(drop=True)
    df_bad = df[df['label'] == 1].reset_index(drop=True)
    
    N_SAMPLES = min(len(df_good), len(df_bad))
    train_val_size = int(N_SAMPLES * 0.8)
    test_size = N_SAMPLES - train_val_size
    
    # Lấy test set
    good_train_val = df_good.sample(n=train_val_size, random_state=42)
    bad_train_val = df_bad.sample(n=train_val_size, random_state=42)
    
    good_test = df_good.drop(good_train_val.index).sample(n=test_size, random_state=42)
    bad_test = df_bad.drop(bad_train_val.index).sample(n=test_size, random_state=42)
    
    df_test = pd.concat([good_test, bad_test]).sample(frac=1, random_state=42)
    
    print(f"\n✅ Đã load {len(df_test):,} samples test")
    print(f"   - Sạch: {len(good_test):,}")
    print(f"   - Độc hại: {len(bad_test):,}")
    
except FileNotFoundError as e:
    print(f"❌ Không tìm thấy file: {e}")
    exit(1)

# Tiền xử lý
print("\n🔧 Đang tiền xử lý dữ liệu test...")
tqdm.pandas(desc="Preprocessing")
df_test['processed_url'] = df_test['url'].progress_apply(preprocess_url_for_keras)

X_test = df_test['processed_url'].values
y_test = df_test['label'].values.astype('float32')

print("✅ Tiền xử lý hoàn tất!")

# ==========================================
# 3. DỰ ĐOÁN
# ==========================================
print("\n🎯 BƯỚC 3: DỰ ĐOÁN TRÊN TEST SET")
print("-"*80)

BATCH_SIZE = 512

print("Đang dự đoán...")
y_pred_proba = model.predict(X_test, batch_size=BATCH_SIZE, verbose=1)
y_pred_proba = y_pred_proba.flatten()
y_pred = (y_pred_proba > 0.5).astype(int)

print("✅ Dự đoán hoàn tất!")

# ==========================================
# 4. TÍNH TOÁN METRICS
# ==========================================
print("\n📊 BƯỚC 4: TÍNH TOÁN METRICS")
print("-"*80)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_pred_proba)
auc_pr = average_precision_score(y_test, y_pred_proba)

print("\n" + "="*80)
print("📈 KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH")
print("="*80)
print(f"Accuracy:           {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precision:          {precision:.4f} ({precision*100:.2f}%)")
print(f"Recall:             {recall:.4f} ({recall*100:.2f}%)")
print(f"F1-Score:           {f1:.4f} ({f1*100:.2f}%)")
print(f"AUC-ROC:            {auc_roc:.4f}")
print(f"Average Precision:  {auc_pr:.4f}")
print("="*80)

# Classification Report
print("\n📋 CLASSIFICATION REPORT:")
print("-"*80)
print(classification_report(y_test, y_pred, 
                          target_names=['Sạch (0)', 'Độc hại (1)'],
                          digits=4))

# Lưu metrics ra CSV
metrics_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC', 'Average Precision'],
    'Value': [f"{accuracy:.4f}", f"{precision:.4f}", f"{recall:.4f}", 
              f"{f1:.4f}", f"{auc_roc:.4f}", f"{auc_pr:.4f}"]
})
metrics_df.to_csv('model_metrics.csv', index=False)
print("\n✅ Đã lưu metrics vào: model_metrics.csv")

# ==========================================
# 5. VẼ CONFUSION MATRIX (cải tiến)
# ==========================================
print("\n🎨 BƯỚC 5: VẼ CÁC BIỂU ĐỒ")
print("-"*80)

cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True,
            xticklabels=['Sạch (Benign)', 'Độc hại (Malicious)'],
            yticklabels=['Sạch (Benign)', 'Độc hại (Malicious)'],
            annot_kws={'size': 16, 'weight': 'bold'})
plt.ylabel('Nhãn Thực Tế (Actual Label)', fontsize=13, fontweight='bold')
plt.xlabel('Nhãn Dự Đoán (Predicted Label)', fontsize=13, fontweight='bold')
plt.title('Ma Trận Nhầm Lẫn (Confusion Matrix)', fontsize=15, fontweight='bold', pad=20)

# Thêm thông tin thống kê
tn, fp, fn, tp = cm.ravel()
stats_text = f'TN={tn:,}  FP={fp:,}\nFN={fn:,}  TP={tp:,}'
plt.text(1, -0.15, stats_text, ha='center', va='top', 
         transform=plt.gca().transAxes, fontsize=11, family='monospace',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

plt.tight_layout()
plt.savefig('confusion_matrix_detailed.png', dpi=300, bbox_inches='tight')
print("✅ Đã lưu: confusion_matrix_detailed.png")
plt.close()

# ==========================================
# 6. ROC CURVE
# ==========================================
fpr, tpr, thresholds_roc = roc_curve(y_test, y_pred_proba)

plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, linewidth=3, label=f'Model (AUC = {auc_roc:.4f})', color='darkblue')
plt.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random Classifier', alpha=0.5)
plt.fill_between(fpr, tpr, alpha=0.2, color='blue')
plt.xlabel('Tỷ Lệ Dương Tính Giả (False Positive Rate)', fontsize=12, fontweight='bold')
plt.ylabel('Tỷ Lệ Dương Tính Thật (True Positive Rate)', fontsize=12, fontweight='bold')
plt.title('Đường Cong ROC (ROC Curve)', fontsize=15, fontweight='bold', pad=20)
plt.legend(fontsize=12, loc='lower right')
plt.grid(alpha=0.3, linestyle='--')
plt.tight_layout()
plt.savefig('roc_curve.png', dpi=300, bbox_inches='tight')
print("✅ Đã lưu: roc_curve.png")
plt.close()

# ==========================================
# 7. PRECISION-RECALL CURVE
# ==========================================
precision_vals, recall_vals, thresholds_pr = precision_recall_curve(y_test, y_pred_proba)

plt.figure(figsize=(10, 8))
plt.plot(recall_vals, precision_vals, linewidth=3, 
         label=f'Model (AP = {auc_pr:.4f})', color='darkgreen')
plt.fill_between(recall_vals, precision_vals, alpha=0.2, color='green')
plt.xlabel('Recall (Độ Nhạy)', fontsize=12, fontweight='bold')
plt.ylabel('Precision (Độ Chính Xác)', fontsize=12, fontweight='bold')
plt.title('Đường Cong Precision-Recall', fontsize=15, fontweight='bold', pad=20)
plt.legend(fontsize=12, loc='lower left')
plt.grid(alpha=0.3, linestyle='--')
plt.tight_layout()
plt.savefig('precision_recall_curve.png', dpi=300, bbox_inches='tight')
print("✅ Đã lưu: precision_recall_curve.png")
plt.close()

# ==========================================
# 8. SCORE DISTRIBUTION
# ==========================================
plt.figure(figsize=(12, 7))
plt.hist(y_pred_proba[y_test == 0], bins=60, alpha=0.65, 
         label='URL Sạch (Benign)', color='green', edgecolor='darkgreen', linewidth=1.2)
plt.hist(y_pred_proba[y_test == 1], bins=60, alpha=0.65, 
         label='URL Độc Hại (Malicious)', color='red', edgecolor='darkred', linewidth=1.2)
plt.axvline(x=0.5, color='black', linestyle='--', linewidth=2.5, label='Ngưỡng (Threshold = 0.5)')
plt.xlabel('Điểm Dự Đoán (Prediction Score)', fontsize=12, fontweight='bold')
plt.ylabel('Tần Số (Frequency)', fontsize=12, fontweight='bold')
plt.title('Phân Bố Điểm Dự Đoán (Score Distribution)', fontsize=15, fontweight='bold', pad=20)
plt.legend(fontsize=12, loc='upper center')
plt.grid(alpha=0.3, axis='y', linestyle='--')
plt.tight_layout()
plt.savefig('score_distribution.png', dpi=300, bbox_inches='tight')
print("✅ Đã lưu: score_distribution.png")
plt.close()

# ==========================================
# 9. THRESHOLD ANALYSIS
# ==========================================
thresholds = np.arange(0.1, 1.0, 0.02)
precisions = []
recalls = []
f1_scores = []
accuracies = []

print("\nĐang phân tích threshold...")
for thresh in tqdm(thresholds, desc="Threshold Analysis"):
    y_pred_thresh = (y_pred_proba >= thresh).astype(int)
    precisions.append(precision_score(y_test, y_pred_thresh))
    recalls.append(recall_score(y_test, y_pred_thresh))
    f1_scores.append(f1_score(y_test, y_pred_thresh))
    accuracies.append(accuracy_score(y_test, y_pred_thresh))

plt.figure(figsize=(12, 7))
plt.plot(thresholds, precisions, label='Precision', marker='o', markersize=4, linewidth=2)
plt.plot(thresholds, recalls, label='Recall', marker='s', markersize=4, linewidth=2)
plt.plot(thresholds, f1_scores, label='F1-Score', marker='^', markersize=4, linewidth=2.5)
plt.plot(thresholds, accuracies, label='Accuracy', marker='d', markersize=4, linewidth=2, linestyle='--')
plt.axvline(x=0.5, color='gray', linestyle='--', alpha=0.7, linewidth=2, label='Mặc Định (0.5)')

# Tìm threshold tối ưu
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]
plt.axvline(x=optimal_threshold, color='red', linestyle=':', linewidth=2.5, 
            label=f'Tối Ưu ({optimal_threshold:.2f})')

plt.xlabel('Ngưỡng Phân Loại (Threshold)', fontsize=12, fontweight='bold')
plt.ylabel('Điểm Số (Score)', fontsize=12, fontweight='bold')
plt.title('Phân Tích Ngưỡng (Threshold Analysis)', fontsize=15, fontweight='bold', pad=20)
plt.legend(fontsize=11, loc='best')
plt.grid(alpha=0.3, linestyle='--')
plt.tight_layout()
plt.savefig('threshold_analysis.png', dpi=300, bbox_inches='tight')
print("✅ Đã lưu: threshold_analysis.png")
plt.close()

print(f"\n🎯 Ngưỡng Tối Ưu: {optimal_threshold:.3f}")
print(f"   F1-Score: {f1_scores[optimal_idx]:.4f}")
print(f"   Precision: {precisions[optimal_idx]:.4f}")
print(f"   Recall: {recalls[optimal_idx]:.4f}")
print(f"   Accuracy: {accuracies[optimal_idx]:.4f}")

# ==========================================
# 10. ERROR ANALYSIS
# ==========================================
print("\n🔍 BƯỚC 6: PHÂN TÍCH LỖI")
print("-"*80)

fp_indices = np.where((y_pred == 1) & (y_test == 0))[0]
fn_indices = np.where((y_pred == 0) & (y_test == 1))[0]
tp_indices = np.where((y_pred == 1) & (y_test == 1))[0]
tn_indices = np.where((y_pred == 0) & (y_test == 0))[0]

print(f"\n📊 Thống Kê Chi Tiết:")
print(f"  ✓ True Positives (TP):   {len(tp_indices):,}")
print(f"  ✓ True Negatives (TN):   {len(tn_indices):,}")
print(f"  ✗ False Positives (FP):  {len(fp_indices):,}")
print(f"  ✗ False Negatives (FN):  {len(fn_indices):,}")

if len(fp_indices) > 0:
    print(f"\n❌ FALSE POSITIVES (Sạch bị nhận là Độc hại):")
    print(f"   Trung bình score: {y_pred_proba[fp_indices].mean():.4f}")
    print(f"   Top 5 examples:")
    for i, idx in enumerate(fp_indices[:5], 1):
        print(f"   {i}. Score: {y_pred_proba[idx]:.4f} | URL: {df_test.iloc[idx]['url'][:80]}")

if len(fn_indices) > 0:
    print(f"\n❌ FALSE NEGATIVES (Độc hại bị nhận là Sạch):")
    print(f"   Trung bình score: {y_pred_proba[fn_indices].mean():.4f}")
    print(f"   Top 5 examples:")
    for i, idx in enumerate(fn_indices[:5], 1):
        print(f"   {i}. Score: {y_pred_proba[idx]:.4f} | URL: {df_test.iloc[idx]['url'][:80]}")

# ==========================================
# 11. BOXPLOT ERROR ANALYSIS
# ==========================================
error_data = {
    'True\nPositive': y_pred_proba[tp_indices] if len(tp_indices) > 0 else [],
    'True\nNegative': y_pred_proba[tn_indices] if len(tn_indices) > 0 else [],
    'False\nPositive': y_pred_proba[fp_indices] if len(fp_indices) > 0 else [],
    'False\nNegative': y_pred_proba[fn_indices] if len(fn_indices) > 0 else []
}

plt.figure(figsize=(12, 7))
box_parts = plt.boxplot([v for v in error_data.values() if len(v) > 0], 
                        labels=[k for k, v in error_data.items() if len(v) > 0],
                        patch_artist=True,
                        notch=True,
                        showmeans=True)

# Tô màu
colors = ['lightgreen', 'lightblue', 'lightcoral', 'lightyellow']
for patch, color in zip(box_parts['boxes'], colors[:len(box_parts['boxes'])]):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

plt.axhline(y=0.5, color='red', linestyle='--', linewidth=2, label='Ngưỡng (0.5)', alpha=0.7)
plt.ylabel('Điểm Dự Đoán (Prediction Score)', fontsize=12, fontweight='bold')
plt.title('Phân Bố Điểm Theo Loại Dự Đoán (Score Distribution by Prediction Type)', 
          fontsize=15, fontweight='bold', pad=20)
plt.legend(fontsize=11)
plt.grid(alpha=0.3, axis='y', linestyle='--')
plt.tight_layout()
plt.savefig('error_boxplot.png', dpi=300, bbox_inches='tight')
print("✅ Đã lưu: error_boxplot.png")
plt.close()

# ==========================================
# 12. SUMMARY TABLE
# ==========================================
summary_data = {
    'Category': ['Tổng Số Mẫu', 'URL Sạch (Thực Tế)', 'URL Độc Hại (Thực Tế)', 
                 'True Positives', 'True Negatives', 
                 'False Positives', 'False Negatives'],
    'Count': [len(y_test), 
              int(np.sum(y_test == 0)), 
              int(np.sum(y_test == 1)),
              len(tp_indices),
              len(tn_indices),
              len(fp_indices),
              len(fn_indices)]
}

summary_df = pd.DataFrame(summary_data)
summary_df.to_csv('evaluation_summary.csv', index=False)

print("\n📊 BẢNG TỔNG HỢP:")
print("-"*80)
print(summary_df.to_string(index=False))

# ==========================================
# 13. TẠO BIỂU ĐỒ SO SÁNH METRICS
# ==========================================
metrics_comparison = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score'],
    'Score': [accuracy, precision, recall, f1]
}

plt.figure(figsize=(10, 6))
bars = plt.bar(metrics_comparison['Metric'], metrics_comparison['Score'], 
               color=['#3498db', '#2ecc71', '#e74c3c', '#f39c12'],
               alpha=0.8, edgecolor='black', linewidth=1.5)

# Thêm giá trị trên mỗi cột
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.4f}\n({height*100:.2f}%)',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.ylabel('Score', fontsize=12, fontweight='bold')
plt.title('So Sánh Các Metrics Đánh Giá', fontsize=15, fontweight='bold', pad=20)
plt.ylim(0, 1.1)
plt.grid(alpha=0.3, axis='y', linestyle='--')
plt.tight_layout()
plt.savefig('metrics_comparison.png', dpi=300, bbox_inches='tight')
print("✅ Đã lưu: metrics_comparison.png")
plt.close()

# ==========================================
# HOÀN THÀNH
# ==========================================
print("\n" + "="*80)
print("🎉 HOÀN THÀNH! TẤT CẢ CÁC ĐỒ THỊ ĐÃ ĐƯỢC TẠO!")
print("="*80)
print("\n📁 Danh sách files đã tạo:")
print("  1. ✅ confusion_matrix_detailed.png")
print("  2. ✅ roc_curve.png")
print("  3. ✅ precision_recall_curve.png")
print("  4. ✅ score_distribution.png")
print("  5. ✅ threshold_analysis.png")
print("  6. ✅ error_boxplot.png")
print("  7. ✅ metrics_comparison.png")
print("  8. ✅ model_metrics.csv")
print("  9. ✅ evaluation_summary.csv")
print("\n💡 Sử dụng các file PNG này để đưa vào báo cáo LaTeX!")
print("="*80)

2025-11-10 07:53:47.062156: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


                    🚀 CODE CỨU VỚT - ĐÁNH GIÁ MÔ HÌNH

📦 BƯỚC 1: LOAD MODEL
--------------------------------------------------------------------------------


I0000 00:00:1762761230.506554    2953 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43742 MB memory:  -> device: 0, name: NVIDIA L40S, pci bus id: 0000:34:00.0, compute capability: 8.9


✅ Đã load model từ: url_classifier_model.keras
✓ Model architecture:


Model: "functional_1"

...

✅ Tiền xử lý hoàn tất!

🎯 BƯỚC 3: DỰ ĐOÁN TRÊN TEST SET
--------------------------------------------------------------------------------
Đang dự đoán...


2025-11-10 07:54:11.056268: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002


2723/2723 ━━━━━━━━━━━━━━━━━━━━ 43s 15ms/step
✅ Dự đoán hoàn tất!

📊 BƯỚC 4: TÍNH TOÁN METRICS
--------------------------------------------------------------------------------

📈 KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH
Accuracy:           0.9826 (98.26%)
Precision:          0.9917 (99.17%)
Recall:             0.9734 (97.34%)
F1-Score:           0.9825 (98.25%)
AUC-ROC:            0.9958
Average Precision:  0.9969

📋 CLASSIFICATION REPORT:
--------------------------------------------------------------------------------
              precision    recall  f1-score   support

    Sạch (0)     0.9739    0.9918    0.9828    697009
 Độc hại (1)     0.9917    0.9734    0.9825    697009

    accuracy                         0.9826   1394018
   macro avg     0.9828    0.9826    0.9826   1394018
weighted avg     0.9828    0.9826    0.9826   1394018


✅ Đã lưu metrics vào: model_metrics.csv

🎨 BƯỚC 5: VẼ CÁC BIỂU ĐỒ
--------------------------------------------------------------------------------
✅ Đã lưu: confusi

Threshold Analysis: 100%|██████████| 45/45 [01:16<00:00,  1.70s/it]


✅ Đã lưu: threshold_analysis.png

🎯 Ngưỡng Tối Ưu: 0.400
   F1-Score: 0.9826
   Precision: 0.9893
   Recall: 0.9759
   Accuracy: 0.9827

🔍 BƯỚC 6: PHÂN TÍCH LỖI
--------------------------------------------------------------------------------

📊 Thống Kê Chi Tiết:
  ✓ True Positives (TP):   678,482
  ✓ True Negatives (TN):   691,309
  ✗ False Positives (FP):  5,700
  ✗ False Negatives (FN):  18,527

❌ FALSE POSITIVES (Sạch bị nhận là Độc hại):
   Trung bình score: 0.7497
   Top 5 examples:
   1. Score: 0.9281 | URL: integrationforall.com/sites/02%20-%20aikido/
   2. Score: 0.5878 | URL: packetstormsecurity.nl/papers/cryptography/ssh-timing.pdf
   3. Score: 0.9264 | URL: roblaquinta.com/wp-admin/js/login_verify2.htm
   4. Score: 0.5071 | URL: twitter.com/TheHotOffice
   5. Score: 0.9934 | URL: whatchadoin.com/penaltybox/

❌ FALSE NEGATIVES (Độc hại bị nhận là Sạch):
   Trung bình score: 0.1347
   Top 5 examples:
   1. Score: 0.3224 | URL: dev.cscslacouronne.org/toutcache/x64/mimidrv.sys
